## Mini Project 2

In [1]:
import requests
import pandas as pd
import sqlite3
import lxml.html
from bs4 import BeautifulSoup

### I'm using soup to connect to one of the few websites available that has an entirely text based table. Surprisingly uncommon, as I've been through twenty tables that have spooky "elements" rather than text that Jupyter doesn't like scraping off of websites.

In [2]:
url = "https://www.dltk-kids.com/poKEmon/pokelist.htm"
r=requests.get(url)

In [3]:
soup = BeautifulSoup(r.content,"html.parser")
table = soup.find('table')
rows = table.find_all('tr')
len(rows)

387

### The count checks out, as there are 386 rows of data as well as a header row.

In [4]:
newTable = []
for row in rows:
    cols = row.find_all('td')
    newTable.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip(), cols[3].text.strip()))

### Now that we've created a table by appending the text to our data, we can convert the table into a Data Frame that we can manipulate.

In [5]:
dfPre = pd.DataFrame(newTable)
dfPre.columns = ["Number", "Pokemon", "Type1", "Type2"]
dfPre.head()

,Number,Pokemon,Type1,Type2
0,#,Pokemon,Type 1,Type 2
1,63,Abra,Psychic,-
2,359,Absol,Dark,-
3,142,Aerodactyl,Rock,Flying
4,306,Aggron,Steel,Rock


### All that needs to be done now is to drop the dummy header row.

In [6]:
df = dfPre.drop(0)
df.head()

,Number,Pokemon,Type1,Type2
1,63,Abra,Psychic,-
2,359,Absol,Dark,-
3,142,Aerodactyl,Rock,Flying
4,306,Aggron,Steel,Rock
5,190,Aipom,Normal,-


### We now have a Data Frame with the requisite 386 rows. With this, we're ready to connect to sqlite and upload our data.

In [7]:
db=sqlite3.connect(":memory:")
cursor=db.cursor()
cursor.execute("""
CREATE TABLE PLIST(NUMBER, POKEMON, TYPE1, TYPE2)
""")

for row in df[df.Type1.isin(["Dark","Steel"])].itertuples():
    print (row[1:])

('359', 'Absol', 'Dark', '-')
('306', 'Aggron', 'Steel', 'Rock')
('304', 'Aron', 'Steel', 'Rock')
('374', 'Beldum', 'Steel', 'Psychic')
('205', 'Foretress', 'Steel', 'Bug')
('229', 'Houndoom', 'Dark', 'Fire')
('228', 'Houndour', 'Dark', 'Fire')
('385', 'Jirachi', 'Steel', 'Psychic')
('305', 'Lairon', 'Steel', 'Rock')
('303', 'Mawile', 'Steel', '-')
('376', 'Metagross', 'Steel', 'Psychic')
('375', 'Metang', 'Steel', 'Psychic')
('262', 'Mightyena', 'Dark', '-')
('198', 'Murkrow', 'Dark', 'Flying')
('261', 'Poochyena', 'Dark', '-')
('379', 'Registeel', 'Steel', '-')
('302', 'Sapleye', 'Dark', 'Ghost')
('227', 'Skarmony', 'Steel', 'Flying')
('215', 'Sneazle\xa0\xa0 \r\n\t\t\t\t\t*fan*', 'Dark', 'Ice')
('208', 'Steelix', 'Steel', 'Ground')
('197', 'Umbreon', 'Dark', '-')
('263', 'Zigzagoon', 'Dark', '-')


In [8]:
DarkSteel = df[df.Type1.isin(["Dark","Steel"])]

In [9]:
for row in DarkSteel.itertuples():
    insertSQLsyntax = """
    INSERT INTO PLIST(NUMBER, POKEMON, TYPE1, TYPE2)
    VALUES (?,?,?,?)
    """
    cursor.execute(insertSQLsyntax, row[1:])
    print(row[1:])
db.commit()

('359', 'Absol', 'Dark', '-')
('306', 'Aggron', 'Steel', 'Rock')
('304', 'Aron', 'Steel', 'Rock')
('374', 'Beldum', 'Steel', 'Psychic')
('205', 'Foretress', 'Steel', 'Bug')
('229', 'Houndoom', 'Dark', 'Fire')
('228', 'Houndour', 'Dark', 'Fire')
('385', 'Jirachi', 'Steel', 'Psychic')
('305', 'Lairon', 'Steel', 'Rock')
('303', 'Mawile', 'Steel', '-')
('376', 'Metagross', 'Steel', 'Psychic')
('375', 'Metang', 'Steel', 'Psychic')
('262', 'Mightyena', 'Dark', '-')
('198', 'Murkrow', 'Dark', 'Flying')
('261', 'Poochyena', 'Dark', '-')
('379', 'Registeel', 'Steel', '-')
('302', 'Sapleye', 'Dark', 'Ghost')
('227', 'Skarmony', 'Steel', 'Flying')
('215', 'Sneazle\xa0\xa0 \r\n\t\t\t\t\t*fan*', 'Dark', 'Ice')
('208', 'Steelix', 'Steel', 'Ground')
('197', 'Umbreon', 'Dark', '-')
('263', 'Zigzagoon', 'Dark', '-')


### Yes, the data is misspelled, has typos everywhere, and features strange hyperlinks in some of the name cells (see Sneazle\xa0...?) , but we still get something usable outputted into our mySQL table. For a sample exercise it works, especially because every other table I tried had elements that didn't play nicely with mySQL's scraping abilities. I've learned you take what you can get.

In [10]:
for row in cursor.execute("""
    SELECT *
    FROM PLIST
    WHERE TYPE2 = 'Flying'
"""):
        print(row)

('198', 'Murkrow', 'Dark', 'Flying')
('227', 'Skarmony', 'Steel', 'Flying')


### That's how we get SQL to print out a list of every Pokemon whose first type is Dark or Steel type and is also part Flying. 

In [11]:
db.close()